<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2024Project/blob/Konstrukcja_detektora_miast/Jak_uzywac_funkcji_do_detekcji_miast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dokument ma na celu wytłumaczenie na przykładzie w jaki sposób korzystać z klasy **City_Geolocalization_Data** oraz jak skonfigurować GitHuba w Colabie.

# Opis klasy

**KONSTRUKTOR** \
**City_Geolocalization_Data(path)** - *path*, to ścieżka do pliku .csv z lokalizacjami.

**FUNKCJE** \
**City_Geolocalization_Data.get_cityname(lon, lat)** \
Zwraca nazwę miasta dla podanych współrzędnych lon i lat. W przypadku braku współrzędnych zwróci None.

**City_Geolocalization_Data.spark_add_geodata(sdf, city = True, state = False, country = False)** \
Zwraca Spark DataFrame, w którym zostają dodane kolumny według przyznanych flag w parametrach funkcji. \
Parametry:  \
*sdf* - Spark DataFrame do którego dodajemy kolumny. \
*city* - Flaga dodania kolumny *City* do DataFramu z nazwą miasta. \
*state* - Flaga dodania kolumny *State* do DataFramu z nazwą stanu. \
*country* - Flaga dodania kolumny *Country* do DataFramu z nazwą kraju.

# Setup repozytorium i środowiska

Najpierw instalacja sparka

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Dalej importujemy nasz projekt do zasobów. Będzie go widać po lewej stronie w zakładce files

In [3]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2024Project.git

fatal: destination path 'BigData2024Project' already exists and is not an empty directory.


*Odkomentować i ustawić odpowiedni branch dla danego kodu*

In [4]:
%cd BigData2024Project/
!git checkout Konstrukcja_detektora_miast
%cd ..

/content/BigData2024Project
Already on 'Konstrukcja_detektora_miast'
Your branch is up to date with 'origin/Konstrukcja_detektora_miast'.
/content


Pakiet poetry jest potrzebny aby zbudować paczkę z naszego projektu big_mess. To jej budowy potrzebny jest plik `pyproject.toml`

In [4]:
!pip install poetry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 44.9 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.0.0
    Uninstalling platformdirs-4.0.0:
      Successfully uninstalled platformdirs-4.0.0
  Attempting uninstall: keyring
    Found existing installation: keyring 23.5.0
    Uninstal

Gdy mamy zainstalowane poetry użyjemy go do zbudowania pakietu

In [5]:
%cd BigData2024Project/
!poetry install
%cd ..

/content/BigData2024Project
Creating virtualenv big-mess-OuyUQzp_-py3.10 in /root/.cache/pypoetry/virtualenvs
Updating dependencies
Resolving dependencies... (0.1s)

Writing lock file

Installing the current project: big_mess (0.1.0)Installing the current project: big_mess (0.1.0)
/content


i jego lokalnego zainstalowania

In [6]:
%cd BigData2024Project/
!pip install -e .
%cd ..

/content/BigData2024Project
Obtaining file:///content/BigData2024Project
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for big-mess (pyproject.toml) ... done
  Created wheel for big-mess: filename=big_mess-0.0.0-0.editable-py3-none-any.whl size=2006 sha256=8ddb9290967fada05577ce68980dbab28a7bccc609242b981be5767b8549f41c
  Stored in directory: /tmp/pip-ephem-wheel-cache-t150drhl/wheels/c6/56/d2/ebcb3f4a28085222be0d9516ae5549cf013abdd1ff392ad062
Successfully built big-mess
/content


Potem występuje trudny krok teraz (prawdopodobnie) trzeba **ZRESTARTOWAĆ RUNTIME**. Wybieramy sekcje Runtime i Restart runtime

Okey - dalej konfigurejemy sparka

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

import findspark
findspark.init()

Wszystko gotowe - można importować

In [6]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, StringType, StructType

from google.colab import drive

from big_mess.city_detector import City_Geolocalization_Data

# Przykład użycia

## 1. Wczytanie danych w sparku

Utworzenie środowiska pyspark do obliczeń:

In [7]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Połączenie z dyskiem:

In [8]:
drive.mount('/content/drive')

Mounted at /content/drive


Wczytanie danych NASA znajdujących się na dysku w sparku:

In [9]:
columns = ['lon', 'lat', 'Date', 'SWdown', 'LWdown', 'SWnet', 'LWnet', 'Qle', 'Qh', 'Qg', 'Qf', 'Snowf', 'Rainf', 'Evap', 'Qs', 'Qsb', 'Qsm', 'AvgSurfT', 'Albedo', 'SWE', 'SnowDepth', 'SnowFrac', 'SoilT_0_10cm', 'SoilT_10_40cm',
           'SoilT_40_100cm', 'SoilT_100_200cm', 'SoilM_0_10cm', 'SoilM_10_40cm', 'SoilM_40_100cm', 'SoilM_100_200cm', 'SoilM_0_100cm', 'SoilM_0_200cm', 'RootMoist', 'SMLiq_0_10cm', 'SMLiq_10_40cm', 'SMLiq_40_100cm', 'SMLiq_100_200cm',
           'SMAvail_0_100cm', 'SMAvail_0_200cm', 'PotEvap', 'ECanop', 'TVeg', 'ESoil', 'SubSnow', 'CanopInt', 'ACond', 'CCond', 'RCS', 'RCT', 'RCQ', 'RCSOL', 'RSmin','RSMacr', 'LAI', 'GVEG', 'Streamflow']

# Utworzenie schematu określającego typ zmiennych
schema = StructType()
for i in columns:
  if i == "Date":
    schema = schema.add(i, StringType(), True)
  else:
    schema = schema.add(i, FloatType(), True)

In [11]:
nasa = spark.read.format('csv').option("header", True).schema(schema).load('/content/drive/MyDrive/BigMess/NASA/NASA.csv')
nasa.show(5)

+---------+-------+------+---------+---------+---------+---------+---------+---------+----------+---+-----+-----+---------+---+----------+---+---------+---------+---+---------+--------+------------+-------------+--------------+---------------+------------+-------------+--------------+---------------+-------------+-------------+----------+------------+-------------+--------------+---------------+---------------+---------------+---------+----------+---------+----------+-------+------------+-----------+------------+----------+----------+----------+----------+---------+------------+---------+----------+----------+
|      lon|    lat|  Date|   SWdown|   LWdown|    SWnet|    LWnet|      Qle|       Qh|        Qg| Qf|Snowf|Rainf|     Evap| Qs|       Qsb|Qsm| AvgSurfT|   Albedo|SWE|SnowDepth|SnowFrac|SoilT_0_10cm|SoilT_10_40cm|SoilT_40_100cm|SoilT_100_200cm|SoilM_0_10cm|SoilM_10_40cm|SoilM_40_100cm|SoilM_100_200cm|SoilM_0_100cm|SoilM_0_200cm| RootMoist|SMLiq_0_10cm|SMLiq_10_40cm|SMLiq_40_100cm|

## 2. Użycie funkcji

Stworzenie obiektu będącego realizacją klasy City_Geolocalization_Data.

In [12]:
geodata_file = "/content/drive/MyDrive/BigMess/NASA/geolokalizacja_wsp_NASA.csv"

geo = City_Geolocalization_Data(geodata_file)

Użycie funkcji *get_cityname*.

In [13]:
geo.get_cityname(-78.0625, 25.0625)

'North Andros'

Użycie funkcji *spark_add_geodata*.

In [21]:
result = geo.spark_add_geodata(nasa, city=True, state=True, country=True)
result.show(5)

/content/spark-3.5.0-bin-hadoop3/python/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


+---------+-------+------+---------+---------+---------+----------+---------+---------+----------+---+-----+-----+---------+---+----------+---+---------+---------+---+---------+--------+------------+-------------+--------------+---------------+------------+-------------+--------------+---------------+-------------+-------------+----------+------------+-------------+--------------+---------------+---------------+---------------+---------+----------+---------+----------+-------+------------+------------+------------+----------+----------+---------+----------+---------+------------+---------+----------+----------+-------------------+-------------------+-------+
|      lon|    lat|  Date|   SWdown|   LWdown|    SWnet|     LWnet|      Qle|       Qh|        Qg| Qf|Snowf|Rainf|     Evap| Qs|       Qsb|Qsm| AvgSurfT|   Albedo|SWE|SnowDepth|SnowFrac|SoilT_0_10cm|SoilT_10_40cm|SoilT_40_100cm|SoilT_100_200cm|SoilM_0_10cm|SoilM_10_40cm|SoilM_40_100cm|SoilM_100_200cm|SoilM_0_100cm|SoilM_0_200cm| Ro

In [22]:
result.select('lon','lat','City','State','Country').filter(result.City != 'NaN').limit(10).toPandas()

,lon,lat,City,State,Country
0,-86.4375,34.6875,Huntsville,Alabama,United States
1,-92.4375,38.9375,Columbia,Missouri,United States
2,-89.5625,39.8125,Springfield,Illinois,United States
3,-77.3125,40.1875,West Pennsboro Township,Pennsylvania,United States
4,-79.4375,40.3125,Unity Township,Pennsylvania,United States
5,-77.6875,41.3125,Noyes Township,Pennsylvania,United States
6,-82.3125,42.8125,St. Clair,Ontario,Canada
7,-79.8125,43.1875,Hamilton,Ontario,Canada
8,-78.1875,44.9375,Highlands East,Ontario,Canada
9,-80.0625,45.1875,The Archipelago Township,Ontario,Canada
